The purpose of this notebook is to contstruct a BLAST database of DJR capsid proteins to use alongside viruscope to identify associated viruses.

In [1]:
import pandas as pd
import os.path as op
import os

In [8]:
tbl = "/mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/nature25474-s4.xlsx"
df = pd.read_excel(tbl, skiprows=[0])

In [9]:
len(df)

14668

In [10]:
df[:1]

,Protein Identifier,Protein- Full Name,Protein - Sequence,Procedural Note,Additional Source Information,Seq Length (aa),mcl_1.5 protein cluster ID,Extended Data Fig. 8 - Panel A - References,Extended Data Fig. 8 - Panel B - Structure,Extended Data Fig. 8 - Panel C - BestHit,...,cdSearch PSSM-ID,cdSearch From,cdSearch To,cdSearch Hit Length,cdSearch E-Value,cdSearch Bitscore,cdSearch Accession,cdSearch Short name,cdSearch Incomplete,cdSearch Superfamily
0,LF_contig_102320_1,>lcl|LF_contig_102320_1 unnamed protein product,FFFNRNPGLYLPLIALQYHEVRVDIDIASDMETYLDKGTFKVWANY...,.,14N.299.NahantLF,250,1,.,Double Jelly Roll,dsDNA_viruses,...,282327.0,53.0,242.0,189.0,1.607730e-44,147.901,pfam04451,Capsid_NCLDV,-,cl04526


In [16]:
with open("/mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/djr_proteins.fasta", "w") as oh:
    for i, l in df.iterrows():
        print(l['Protein- Full Name'], file=oh)
        
        for j in range(0,len(l['Protein - Sequence']), 80):
            print(l['Protein - Sequence'][j:j+80].replace("*",""), file=oh)

In [18]:
!formatdb

/mnt/scgc_nfs/opt/common/bcl2fastq/2.17.1.14/bin/sh: formatdb: command not found


In [19]:
!makeblastdb -in /mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/djr_proteins.fasta -dbtype prot



Building a new DB, current time: 03/07/2018 11:47:18
New DB name:   /mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/djr_proteins.fasta
New DB title:  /mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/djr_proteins.fasta
Sequence type: Protein
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 14668 sequences in 0.774198 seconds.


In [20]:
proteins = "/mnt/scgc/simon/simonsproject/bats248_vs/batch_prots.fasta"

In [22]:
!grep -c ">" {proteins}

4508644


In [23]:
!cat /mnt/scgc/simon/simonsproject/bats248_vs/clustering/new_seeds.fasta /mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90.fasta > /mnt/scgc/simon/simonsproject/bats248_vs/clustering/all_seeds.fasta

In [24]:
os.mkdir("/mnt/scgc/simon/simonsproject/bats248_vs/clustering/djr/")

In [25]:
query = '/mnt/scgc/simon/simonsproject/bats248_vs/clustering/all_seeds.fasta'
db = '/mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/djr_proteins.fasta'
out = "/mnt/scgc/simon/simonsproject/bats248_vs/clustering/djr/all_seeds_vs_djr.out"
# blastn -query fasta.file -db nr -outfmt 6 -num_alignments 1 -num_descriptions 1 -out haktan.txt -dust no -task blastn


In [31]:
blastp = 'blastp -query {query} -db {db} -outfmt 6 -out {out} -num_threads 20'.format(query=query, db=db, out=out)

In [34]:
print("echo '{blastp}'|qsub -N blastp -q route -l walltime=10:00:00,ncpus=20,mem=150G -j oe -o /home/julia/out/180307_djr_blast.out".format(blastp=blastp))

echo 'blastp -query /mnt/scgc/simon/simonsproject/bats248_vs/clustering/all_seeds.fasta -db /mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/djr_proteins.fasta -outfmt 6 -out /mnt/scgc/simon/simonsproject/bats248_vs/clustering/djr/all_seeds_vs_djr.out -num_threads 20'|qsub -N blastp -q route -l walltime=10:00:00,ncpus=20,mem=150G -j oe -o /home/julia/out/180307_djr_blast.out
